# 1. Importo librerias

In [2]:
from binance.client import Client
from datetime import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import plot_heatmaps

import multiprocessing
import pandas as pd

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# 2. Configuro ambiente y traigo datos historicos

In [3]:
## necesario por cuestiones de optimizacion de performance de la simulacion
multiprocessing.set_start_method('fork', force=True)

In [126]:
#TODO: mover a variables de entorno
api_key = ''
secret_key = ''

client = Client(api_key, secret_key)


# bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '14 jun, 2021', '17 jun, 2021')
bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '60 days ago')


def cast_string_to_float(data):
    if type(data) == str:
        casted_data = float(data)
    else:
        casted_data = datetime.fromtimestamp(data/1000.0)
    return casted_data

# delete unwanted data - just keep date, open, high, low, close
formated_bars = []
for line in bars:
    del line[6:]
    # cast string numbers to integer
    formated_bars.append(list(map(cast_string_to_float, line)))

#  store data in a dataframe
btc_df = pd.DataFrame(formated_bars, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
btc_df.set_index('Date', inplace=True)
# Converting the index as date
btc_df.index = pd.to_datetime(btc_df.index)
btc_df

,Open,High,Low,Close,Volume
Date,,,,,
2021-05-10 18:26:00,55444.93,55466.46,55390.57,55439.56,37.039448
2021-05-10 18:27:00,55439.56,55518.23,55366.27,55451.88,51.192535
2021-05-10 18:28:00,55446.76,55451.96,55350.04,55384.08,46.374217
2021-05-10 18:29:00,55382.78,55389.99,55350.00,55371.37,39.271932
2021-05-10 18:30:00,55371.37,55409.22,55360.00,55396.05,22.218021
...,...,...,...,...,...
2021-07-09 21:23:00,34021.64,34027.64,34017.42,34022.99,10.853364
2021-07-09 21:24:00,34022.99,34027.62,33993.08,34004.23,22.838081
2021-07-09 21:25:00,34005.83,34022.85,33993.06,33996.18,15.727294


# 3. Armo una estrategia de tendencia con AROON


## 3.1 Estrategia

In [263]:
from talib import AROON, SMA, NATR
import pandas as pd

class AroonPullback(Strategy):

    AROON_LENGTH = 30
    # PULLBACK_MAX_LENGTH = 10
    VOLUME_LENGTH = 10
    VOLUME_THRESHOLD = 15
    SMA_LENGTH = 15
    # TREND_FOLLOW_LENGTH = 3
    # NET_VOLUME_THRESHOLD = 0
    NATR_THRESHOLD_MR = 37
    BULL_BEAR_MARKET_EMA_THRESHOLD = 300
    # NATR_THRESHOLD_TF = 1000

    # def STD_DEV(self, array, n):
    #    return pd.Series(array).rolling(n).std()
    
    def VOLUME_AVG(self, volume):
       return pd.Series(volume).rolling(self.VOLUME_LENGTH).mean()

    def EMA_AVG(self, data):
        return pd.Series(data).ewm(span=self.BULL_BEAR_MARKET_EMA_THRESHOLD).mean()

    def init(self):
        self.aroondown, self.aroonup = self.I(AROON, self.data.High, self.data.Low, timeperiod = self.AROON_LENGTH)
        self.volume_avg = self.I(self.VOLUME_AVG, self.data.Volume)
        self.mean_price = self.I(SMA, self.data.Close, timeperiod=self.SMA_LENGTH)
        # self.countdown = self.PULLBACK_MAX_LENGTH
        self.natr = self.I(NATR, self.data.High, self.data.Low, self.data.Close, timeperiod = 2)
        self.ema = self.I(self.EMA_AVG, self.data.Close)
        # self.has_market_panicked = False

        # self.std_dev = self.I(self.STD_DEV, self.data.Close, self.STANDARD_DEVIATION_LENGTH)

    def next(self):
        if (not self.position and self.natr > (self.NATR_THRESHOLD_MR/100)):
            # if (self.aroonup == 100 and self.data.Volume > (self.VOLUME_THRESHOLD / 10 * self.volume_avg) and self.data.Volume[-2] < self.volume_avg[-2]):
            if (self.aroonup == 100 and self.data.Volume > (self.VOLUME_THRESHOLD / 10 * self.volume_avg) and self.isMarketDowntrending()):
                self.sell()
                # if (self.natr < self.NATR_THRESHOLD_TF):
                    # self.sell()
                    # self.countdown = self.PULLBACK_MAX_LENGTH
                # else:
                    # self.buy()
                    # self.countdown = self.TREND_FOLLOW_LENGTH
                    # self.has_market_panicked = True
            # elif(self.aroondown == 100 and self.data.Volume > (self.VOLUME_THRESHOLD / 10 * self.volume_avg) and self.data.Volume[-2] < self.volume_avg[-2]):
            elif(self.aroondown == 100 and self.data.Volume > (self.VOLUME_THRESHOLD / 10 * self.volume_avg) and self.isMarketUptrending()):
                self.buy()
                # if (self.natr < self.NATR_THRESHOLD_TF):
                #     self.buy()
                    # self.countdown = self.PULLBACK_MAX_LENGTH
                # else:
                #     self.sell()
                    # self.countdown = self.TREND_FOLLOW_LENGTH
                    # self.has_market_panicked = True
        else:
            # self.countdown -= 1
            # if (self.countdown == 0):
            #     self.position.close()
            #     self.has_market_panicked = False
            # if (self.has_market_panicked):
            #     if (self.position.is_long and self.data.Close <= self.mean_price):
            #         self.position.close()
            #         self.has_market_panicked = False
            #     elif (self.position.is_short and self.data.Close >= self.mean_price):
            #         self.position.close()
            #         self.has_market_panicked = False
            # else:
            if (self.position.is_long and self.data.Close >= self.mean_price):
                self.position.close()
            elif (self.position.is_short and self.data.Close <= self.mean_price):
                self.position.close()

    def isMarketDowntrending(self):
        return (self.data.Close < self.ema)

    def isMarketUptrending(self):
        return (self.data.Close > self.ema)
                


## 3.2 Resultados

In [274]:
bt = Backtest(btc_df, AroonPullback, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt.run()
stats

Start                     2021-05-10 18:26:00
End                       2021-07-09 21:27:00
Duration                     60 days 03:01:00
Exposure Time [%]                     2.27761
Equity Final [$]               1385201.169704
Equity Peak [$]                1386351.279704
Return [%]                          38.520117
Buy & Hold Return [%]              -38.561976
Return (Ann.) [%]                  602.679389
Volatility (Ann.) [%]              189.422904
Sharpe Ratio                         3.181661
Sortino Ratio                       57.839099
Calmar Ratio                       112.615199
Max. Drawdown [%]                    -5.35167
Avg. Drawdown [%]                   -0.593624
Max. Drawdown Duration       11 days 12:51:00
Avg. Drawdown Duration        0 days 10:54:00
# Trades                                  184
Win Rate [%]                        72.826087
Best Trade [%]                       3.103065
Worst Trade [%]                     -3.894395
Avg. Trade [%]                    

In [239]:
bt.plot()



/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '10T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


In [192]:
stats._trades.sort_values('ReturnPct').tail(20)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
72,-386,15120,15122,2462.114760,2440.93,8177.317360,0.008604,2021-05-21 21:59:00,2021-05-21 22:01:00,0 days 00:02:00
330,-551,65493,65499,1839.264000,1822.73,9110.234000,0.008989,2021-06-25 21:32:00,2021-06-25 21:38:00,0 days 00:06:00
87,-486,17669,17676,1991.742984,1973.81,8715.430224,0.009004,2021-05-23 16:28:00,2021-05-23 16:35:00,0 days 00:07:00
8,-248,2189,2193,3978.727872,3942.26,9044.032256,0.009166,2021-05-12 22:28:00,2021-05-12 22:32:00,0 days 00:04:00
299,-497,59288,59291,1950.329556,1931.64,9288.709332,0.009583,2021-06-21 14:07:00,2021-06-21 14:10:00,0 days 00:03:00
96,372,19078,19087,2540.655856,2565.16,9115.541568,0.009645,2021-05-24 15:57:00,2021-05-24 16:06:00,0 days 00:09:00
79,-402,16207,16212,2319.161964,2295.41,9548.289528,0.010242,2021-05-22 16:06:00,2021-05-22 16:11:00,0 days 00:05:00
143,-405,26105,26108,2334.465840,2310.40,9746.665200,0.010309,2021-05-29 13:04:00,2021-05-29 13:07:00,0 days 00:03:00
101,363,19453,19459,2683.242868,2712.33,10558.628916,0.010840,2021-05-24 22:12:00,2021-05-24 22:18:00,0 days 00:06:00
70,-382,14975,14984,2426.568984,2399.23,10443.491888,0.011267,2021-05-21 19:34:00,2021-05-21 19:43:00,0 days 00:09:00


In [278]:
trades_btc = stats._trades
trades_btc['type'] = 0

trades_btc

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,type
0,-19,3069,3075,50433.588492,49944.47,9293.251348,0.009698,2021-05-12 21:35:00,2021-05-12 21:41:00,0 days 00:06:00,0
1,-19,3158,3175,50569.744008,50504.89,1232.226152,0.001282,2021-05-12 23:04:00,2021-05-12 23:21:00,0 days 00:17:00,0
2,-20,3274,3290,50441.805204,50800.00,-7163.895920,-0.007101,2021-05-13 01:00:00,2021-05-13 01:16:00,0 days 00:16:00,0
3,-19,3650,3655,50171.173500,49887.73,5385.426500,0.005650,2021-05-13 07:16:00,2021-05-13 07:21:00,0 days 00:05:00,0
4,-20,3725,3737,49611.937284,49584.19,554.945680,0.000559,2021-05-13 08:31:00,2021-05-13 08:43:00,0 days 00:12:00,0
...,...,...,...,...,...,...,...,...,...,...,...
179,38,71437,71442,35415.910700,35722.66,11656.473400,0.008661,2021-06-29 09:03:00,2021-06-29 09:08:00,0 days 00:05:00,0
180,-41,74497,74508,33409.880700,33439.00,-1193.891300,-0.000872,2021-07-01 12:03:00,2021-07-01 12:14:00,0 days 00:11:00,0
181,-40,81526,81533,34143.097296,33992.57,6021.091840,0.004409,2021-07-06 09:12:00,2021-07-06 09:19:00,0 days 00:07:00,0
182,39,82712,82717,34648.593896,34792.07,5595.568056,0.004141,2021-07-07 04:58:00,2021-07-07 05:03:00,0 days 00:05:00,0


## 3.3 Optimizacion

### Optimizacion EMA


In [75]:
%%time
stats, heatmap = bt.optimize(BULL_BEAR_MARKET_EMA_THRESHOLD = range(185,901,5), maximize='Return [%]',return_heatmap=True)

CPU times: user 6.05 s, sys: 236 ms, total: 6.28 s
Wall time: 12min


In [79]:
stats

Start                     2021-05-10 17:00:00
End                       2021-07-09 20:01:00
Duration                     60 days 03:01:00
Exposure Time [%]                    2.314569
Equity Final [$]               1364527.308192
Equity Peak [$]                1364527.308192
Return [%]                          36.452731
Buy & Hold Return [%]              -38.790955
Return (Ann.) [%]                  542.215179
Volatility (Ann.) [%]              174.555218
Sharpe Ratio                         3.106267
Sortino Ratio                       82.872904
Calmar Ratio                       119.964073
Max. Drawdown [%]                   -4.519813
Avg. Drawdown [%]                   -0.526369
Max. Drawdown Duration        8 days 07:17:00
Avg. Drawdown Duration        0 days 11:36:00
# Trades                                  177
Win Rate [%]                        69.491525
Best Trade [%]                       1.772976
Worst Trade [%]                     -2.856759
Avg. Trade [%]                    

In [80]:
stats._strategy

<Strategy AroonPullback(BULL_BEAR_MARKET_EMA_THRESHOLD=835)>

In [81]:
heatmap.sort_values().tail(10)

BULL_BEAR_MARKET_EMA_THRESHOLD
775                               35.917866
760                               36.037637
755                               36.037637
750                               36.037637
765                               36.098765
830                               36.367655
845                               36.452731
850                               36.452731
835                               36.452731
840                               36.452731
Name: Return [%], dtype: float64

### Optimizacion volumen

In [230]:
%%time
stats, heatmap = bt.optimize(VOLUME_THRESHOLD = range(10,31, 5), VOLUME_LENGTH = range(5,20,5), return_heatmap=True)

CPU times: user 8.21 s, sys: 266 ms, total: 8.48 s
Wall time: 1min 38s


In [231]:
stats

Start                     2021-05-10 18:26:00
End                       2021-07-09 21:27:00
Duration                     60 days 03:01:00
Exposure Time [%]                    2.544409
Equity Final [$]               1268568.395116
Equity Peak [$]                1269200.011508
Return [%]                           26.85684
Buy & Hold Return [%]              -38.561976
Return (Ann.) [%]                   315.13529
Volatility (Ann.) [%]               102.58427
Sharpe Ratio                         3.071965
Sortino Ratio                       28.889541
Calmar Ratio                        47.171073
Max. Drawdown [%]                   -6.680689
Avg. Drawdown [%]                   -0.515733
Max. Drawdown Duration       11 days 21:58:00
Avg. Drawdown Duration        0 days 14:20:00
# Trades                                  198
Win Rate [%]                         70.20202
Best Trade [%]                       1.772976
Worst Trade [%]                     -3.894395
Avg. Trade [%]                    

In [233]:
stats._strategy



<Strategy AroonPullback(VOLUME_THRESHOLD=20,VOLUME_LENGTH=15)>

In [555]:
heatmap.sort_values().tail(10)

VOLUME_THRESHOLD  VOLUME_LENGTH
30                20              NaN
Name: SQN, dtype: float64

In [232]:
plot_heatmaps(heatmap, agg='mean')

Column(id='28492', ...)

### Optimizacion SMA reversion


In [249]:
%%time
stats, heatmap = bt.optimize(SMA_LENGTH = range(7,20), return_heatmap=True)

CPU times: user 8.17 s, sys: 310 ms, total: 8.47 s
Wall time: 1min 35s


In [250]:
stats

Start                     2021-05-10 18:26:00
End                       2021-07-09 21:27:00
Duration                     60 days 03:01:00
Exposure Time [%]                    3.139221
Equity Final [$]                 1327684.3747
Equity Peak [$]                1333401.090856
Return [%]                          32.768437
Buy & Hold Return [%]              -38.561976
Return (Ann.) [%]                  445.194061
Volatility (Ann.) [%]              122.612179
Sharpe Ratio                         3.630912
Sortino Ratio                       68.509995
Calmar Ratio                       104.518865
Max. Drawdown [%]                   -4.259461
Avg. Drawdown [%]                   -0.631605
Max. Drawdown Duration       11 days 08:54:00
Avg. Drawdown Duration        0 days 12:16:00
# Trades                                  296
Win Rate [%]                        68.918919
Best Trade [%]                       3.103065
Worst Trade [%]                     -2.846142
Avg. Trade [%]                    

In [254]:
stats._strategy

<Strategy AroonPullback>

In [255]:
heatmap.sort_values()

SMA_LENGTH
14            2.077106
8             2.153474
7             2.184505
17            2.198282
19            2.214328
18            2.361965
10            2.399082
9             2.475206
15            2.611547
11            2.647393
13            2.647414
16            2.817012
12            2.882449
Name: SQN, dtype: float64

### Optimizacion NATR de activacion

In [245]:
%%time
stats, heatmap = bt.optimize(NATR_THRESHOLD_MR = range(25,50), maximize='Return [%]', return_heatmap=True)

CPU times: user 7.98 s, sys: 282 ms, total: 8.26 s
Wall time: 2min 8s


In [246]:
stats

Start                     2021-05-10 18:26:00
End                       2021-07-09 21:27:00
Duration                     60 days 03:01:00
Exposure Time [%]                     2.27761
Equity Final [$]               1385201.169704
Equity Peak [$]                1386351.279704
Return [%]                          38.520117
Buy & Hold Return [%]              -38.561976
Return (Ann.) [%]                  602.679389
Volatility (Ann.) [%]              189.422904
Sharpe Ratio                         3.181661
Sortino Ratio                       57.839099
Calmar Ratio                       112.615199
Max. Drawdown [%]                    -5.35167
Avg. Drawdown [%]                   -0.593624
Max. Drawdown Duration       11 days 12:51:00
Avg. Drawdown Duration        0 days 10:54:00
# Trades                                  184
Win Rate [%]                        72.826087
Best Trade [%]                       3.103065
Worst Trade [%]                     -3.894395
Avg. Trade [%]                    

In [247]:
stats._strategy

<Strategy AroonPullback(NATR_THRESHOLD_MR=37)>

In [248]:
heatmap.sort_values().tail(10)

NATR_THRESHOLD_MR
28                   32.503853
33                   32.686558
30                   33.050827
39                   33.669472
29                   33.955246
27                   34.045613
38                   35.659607
35                   38.174935
36                   38.446360
37                   38.520117
Name: Return [%], dtype: float64

### Optimizacion de ventana de Aroon

In [50]:
%%time
stats, heatmap = bt.optimize(AROON_LENGTH = range(5,60,5), return_heatmap=True)

CPU times: user 13.1 s, sys: 476 ms, total: 13.6 s
Wall time: 2min 10s


In [51]:
stats

Start                     2021-04-10 15:40:00
End                       2021-07-09 18:43:00
Duration                     90 days 03:03:00
Exposure Time [%]                    0.637804
Equity Final [$]               1187978.711492
Equity Peak [$]                1188997.886992
Return [%]                          18.797871
Buy & Hold Return [%]              -43.328821
Return (Ann.) [%]                   99.552285
Volatility (Ann.) [%]               21.257616
Sharpe Ratio                         4.683135
Sortino Ratio                       36.550578
Calmar Ratio                        43.871848
Max. Drawdown [%]                   -2.269161
Avg. Drawdown [%]                   -0.466214
Max. Drawdown Duration       13 days 18:16:00
Avg. Drawdown Duration        0 days 20:17:00
# Trades                                   83
Win Rate [%]                        68.674699
Best Trade [%]                       1.064826
Worst Trade [%]                     -0.916845
Avg. Trade [%]                    

In [54]:
stats._strategy

<Strategy AroonPullback(AROON_LENGTH=30)>

In [55]:
heatmap.sort_values()

AROON_LENGTH
5               1.119235
45              2.676716
40              2.688224
55              2.689207
50              2.840747
15              2.900761
25              3.042497
10              3.269772
20              3.406104
35              3.545709
30              4.343172
Name: SQN, dtype: float64

## 4. Pruebo con altcoins

In [271]:
#TODO: mover a variables de entorno
api_key = ''
secret_key = ''

client = Client(api_key, secret_key)


# bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '14 jun, 2021', '17 jun, 2021')
bars = client.get_historical_klines('XRPUSDT', Client.KLINE_INTERVAL_1MINUTE, '60 days ago')


def cast_string_to_float(data):
    if type(data) == str:
        casted_data = float(data)
    else:
        casted_data = datetime.fromtimestamp(data/1000.0)
    return casted_data

# delete unwanted data - just keep date, open, high, low, close
formated_bars = []
for line in bars:
    del line[6:]
    # cast string numbers to integer
    formated_bars.append(list(map(cast_string_to_float, line)))

#  store data in a dataframe
xrp_df = pd.DataFrame(formated_bars, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
xrp_df.set_index('Date', inplace=True)
# Converting the index as date
xrp_df.index = pd.to_datetime(xrp_df.index)
xrp_df

,Open,High,Low,Close,Volume
Date,,,,,
2021-05-11 12:59:00,1.4550,1.4562,1.4519,1.4553,802929.72
2021-05-11 13:00:00,1.4557,1.4579,1.4546,1.4557,831144.58
2021-05-11 13:01:00,1.4559,1.4562,1.4512,1.4514,751192.31
2021-05-11 13:02:00,1.4515,1.4526,1.4458,1.4480,1169671.58
2021-05-11 13:03:00,1.4483,1.4634,1.4483,1.4629,1038878.67
...,...,...,...,...,...
2021-07-10 15:57:00,0.6172,0.6172,0.6169,0.6170,70658.36
2021-07-10 15:58:00,0.6171,0.6175,0.6170,0.6175,90782.38
2021-07-10 15:59:00,0.6175,0.6185,0.6175,0.6182,142495.56


In [279]:
bt_2 = Backtest(eth_df, AroonPullback, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt_2.run()
stats

Start                     2021-05-11 09:59:00
End                       2021-07-10 13:00:00
Duration                     60 days 03:01:00
Exposure Time [%]                    3.360976
Equity Final [$]               1295083.571728
Equity Peak [$]                1313340.050236
Return [%]                          29.508357
Buy & Hold Return [%]              -46.615276
Return (Ann.) [%]                   369.83549
Volatility (Ann.) [%]              134.582455
Sharpe Ratio                         2.748022
Sortino Ratio                       28.181807
Calmar Ratio                        62.062087
Max. Drawdown [%]                   -5.959121
Avg. Drawdown [%]                   -0.847798
Max. Drawdown Duration       23 days 20:36:00
Avg. Drawdown Duration        0 days 14:42:00
# Trades                                  253
Win Rate [%]                        66.007905
Best Trade [%]                       1.904153
Worst Trade [%]                     -2.706334
Avg. Trade [%]                    

In [280]:
trades_eth = stats._trades
trades_eth['type'] = 1
trades_eth

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,type
0,-240,1682,1695,4160.115288,4170.82,-2569.130880,-0.002573,2021-05-12 14:01:00,2021-05-12 14:14:00,0 days 00:13:00,1
1,-242,1863,1872,4104.837408,4075.45,7111.752736,0.007159,2021-05-12 17:02:00,2021-05-12 17:11:00,0 days 00:09:00,1
2,-252,2189,2193,3978.727872,3942.26,9189.903744,0.009166,2021-05-12 22:28:00,2021-05-12 22:32:00,0 days 00:04:00,1
3,-257,2341,2352,3939.243672,3952.69,-3455.706296,-0.003413,2021-05-13 01:00:00,2021-05-13 01:11:00,0 days 00:11:00,1
4,-270,2792,2798,3733.945824,3713.50,5520.372480,0.005476,2021-05-13 08:31:00,2021-05-13 08:37:00,0 days 00:06:00,1
...,...,...,...,...,...,...,...,...,...,...,...
248,617,73801,73813,2113.875212,2109.22,-2872.265804,-0.002202,2021-07-01 16:00:00,2021-07-01 16:12:00,0 days 00:12:00,1
249,-573,78790,78808,2269.391880,2283.27,-7952.162760,-0.006115,2021-07-05 03:09:00,2021-07-05 03:27:00,0 days 00:18:00,1
250,-585,79499,79511,2211.125196,2218.65,-4402.010340,-0.003403,2021-07-05 14:58:00,2021-07-05 15:10:00,0 days 00:12:00,1
251,554,80403,80411,2324.929600,2335.00,5579.001600,0.004331,2021-07-06 06:02:00,2021-07-06 06:10:00,0 days 00:08:00,1


In [181]:
# ---------------- ETHUSDT --------------------
# Return [%]                          29.508357
# Buy & Hold Return [%]              -46.615276
# Return (Ann.) [%]                   369.83549
# Volatility (Ann.) [%]              134.582455
# Sharpe Ratio                         2.748022
# Sortino Ratio                       28.181807
# Calmar Ratio                        62.062087
# Max. Drawdown [%]                   -5.959121
# Avg. Drawdown [%]                   -0.847798
# Max. Drawdown Duration       23 days 20:36:00
# Avg. Drawdown Duration        0 days 14:42:00
# # Trades                                  253
# Win Rate [%]                        66.007905
# Best Trade [%]                       1.904153
# Worst Trade [%]                     -2.706334
# Avg. Trade [%]                       0.102406
# Max. Trade Duration           0 days 00:45:00
# Avg. Trade Duration           0 days 00:11:00
# Profit Factor                        1.485939
# Expectancy [%]                       0.104764
# SQN                                  2.337261

# ---------------- EOSUSDT --------------------
# Return [%]                          18.193087
# Buy & Hold Return [%]              -64.953173
# Return (Ann.) [%]                  224.812111
# Volatility (Ann.) [%]              112.250998
# Sharpe Ratio                         2.002763
# Sortino Ratio                       10.986823
# Calmar Ratio                        22.079498
# Max. Drawdown [%]                  -10.181939
# Avg. Drawdown [%]                   -1.282827
# Max. Drawdown Duration       23 days 02:22:00
# Avg. Drawdown Duration        0 days 23:07:00
# # Trades                                  387
# Win Rate [%]                        66.666667
# Best Trade [%]                       1.892271
# Worst Trade [%]                     -4.957256
# Avg. Trade [%]                       0.043201
# Max. Trade Duration           0 days 00:55:00
# Avg. Trade Duration           0 days 00:11:00
# Profit Factor                        1.179888
# Expectancy [%]                       0.046481
# SQN                                   1.04515

# ---------------- AXSUSDT --------------------
# Return [%]                          -7.783136
# Buy & Hold Return [%]              132.519478
# Return (Ann.) [%]                  -35.096962
# Volatility (Ann.) [%]               53.851152
# Sharpe Ratio                              0.0
# Sortino Ratio                             0.0
# Calmar Ratio                              0.0
# Max. Drawdown [%]                  -33.183431
# Avg. Drawdown [%]                   -1.992953
# Max. Drawdown Duration       52 days 03:37:00
# Avg. Drawdown Duration        2 days 07:17:00
# # Trades                                  533
# Win Rate [%]                        64.165103
# Best Trade [%]                       3.384944
# Worst Trade [%]                    -13.230799
# Avg. Trade [%]                      -0.015201
# Max. Trade Duration           0 days 00:56:00
# Avg. Trade Duration           0 days 00:11:00
# Profit Factor                        0.982331
# Expectancy [%]                      -0.006385
# SQN                                 -0.271708

# ---------------- XRPUSDT --------------------



In [217]:
bt_2.plot()

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '10T'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


In [190]:
stats._trades.sort_values('ReturnPct').tail(20)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
72,-386,15120,15122,2462.114760,2440.93,8177.317360,0.008604,2021-05-21 21:59:00,2021-05-21 22:01:00,0 days 00:02:00
330,-551,65493,65499,1839.264000,1822.73,9110.234000,0.008989,2021-06-25 21:32:00,2021-06-25 21:38:00,0 days 00:06:00
87,-486,17669,17676,1991.742984,1973.81,8715.430224,0.009004,2021-05-23 16:28:00,2021-05-23 16:35:00,0 days 00:07:00
8,-248,2189,2193,3978.727872,3942.26,9044.032256,0.009166,2021-05-12 22:28:00,2021-05-12 22:32:00,0 days 00:04:00
299,-497,59288,59291,1950.329556,1931.64,9288.709332,0.009583,2021-06-21 14:07:00,2021-06-21 14:10:00,0 days 00:03:00
96,372,19078,19087,2540.655856,2565.16,9115.541568,0.009645,2021-05-24 15:57:00,2021-05-24 16:06:00,0 days 00:09:00
79,-402,16207,16212,2319.161964,2295.41,9548.289528,0.010242,2021-05-22 16:06:00,2021-05-22 16:11:00,0 days 00:05:00
143,-405,26105,26108,2334.465840,2310.40,9746.665200,0.010309,2021-05-29 13:04:00,2021-05-29 13:07:00,0 days 00:03:00
101,363,19453,19459,2683.242868,2712.33,10558.628916,0.010840,2021-05-24 22:12:00,2021-05-24 22:18:00,0 days 00:06:00
70,-382,14975,14984,2426.568984,2399.23,10443.491888,0.011267,2021-05-21 19:34:00,2021-05-21 19:43:00,0 days 00:09:00


## 5. Optimizo altcoins


### ETHUSDT


In [253]:
bt_2 = Backtest(eth_df, AroonPullback, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt_2.run()
stats

Start                     2021-05-11 09:59:00
End                       2021-07-10 13:00:00
Duration                     60 days 03:01:00
Exposure Time [%]                    4.983715
Equity Final [$]               1345736.006364
Equity Peak [$]                 1366693.65576
Return [%]                          34.573601
Buy & Hold Return [%]              -46.615276
Return (Ann.) [%]                  478.305864
Volatility (Ann.) [%]              191.513219
Sharpe Ratio                         2.497508
Sortino Ratio                       29.096883
Calmar Ratio                        48.163927
Max. Drawdown [%]                    -9.93079
Avg. Drawdown [%]                   -0.704388
Max. Drawdown Duration       28 days 00:28:00
Avg. Drawdown Duration        0 days 11:21:00
# Trades                                  380
Win Rate [%]                        65.789474
Best Trade [%]                       1.904153
Worst Trade [%]                     -2.706334
Avg. Trade [%]                    

### Bull Bear Threshold

In [202]:
%%time
stats, heatmap = bt_2.optimize(BULL_BEAR_MARKET_EMA_THRESHOLD = range(30,901,5), maximize='Return [%]',return_heatmap=True)

CPU times: user 9.12 s, sys: 302 ms, total: 9.42 s
Wall time: 14min 50s


In [203]:
stats

Start                     2021-05-11 09:59:00
End                       2021-07-10 13:00:00
Duration                     60 days 03:01:00
Exposure Time [%]                    3.163475
Equity Final [$]               1199573.537272
Equity Peak [$]                1203228.077264
Return [%]                          19.957354
Buy & Hold Return [%]              -46.615276
Return (Ann.) [%]                  190.655265
Volatility (Ann.) [%]               63.997778
Sharpe Ratio                         2.979092
Sortino Ratio                       16.123958
Calmar Ratio                        35.147038
Max. Drawdown [%]                   -5.424504
Avg. Drawdown [%]                   -0.733578
Max. Drawdown Duration       22 days 16:35:00
Avg. Drawdown Duration        0 days 16:14:00
# Trades                                  231
Win Rate [%]                        64.069264
Best Trade [%]                        1.39054
Worst Trade [%]                     -2.706334
Avg. Trade [%]                    

In [204]:
stats._strategy

<Strategy AroonPullback(BULL_BEAR_MARKET_EMA_THRESHOLD=300)>

In [207]:
heatmap.sort_values()

BULL_BEAR_MARKET_EMA_THRESHOLD
85                                -1.287782
30                                 0.424181
35                                 0.424181
80                                 0.844026
40                                 0.903006
                                    ...    
295                               18.945044
310                               19.083126
280                               19.169007
305                               19.261380
300                               19.957354
Name: Return [%], Length: 175, dtype: float64

In [205]:
stats._trades.sort_values('ReturnPct')

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
28,374,11826,11844,2720.947944,2647.31,-27540.591056,-0.027063,2021-05-19 15:05:00,2021-05-19 15:23:00,0 days 00:18:00
13,-274,6428,6451,3698.330076,3790.00,-25117.559176,-0.024787,2021-05-15 21:07:00,2021-05-15 21:30:00,0 days 00:23:00
103,-440,30269,30292,2581.886832,2629.92,-21134.593920,-0.018604,2021-06-01 10:28:00,2021-06-01 10:51:00,0 days 00:23:00
139,-445,42992,43022,2520.581364,2561.92,-18395.693020,-0.016400,2021-06-10 06:31:00,2021-06-10 07:01:00,0 days 00:30:00
32,375,12730,12755,2648.799096,2610.76,-14264.661000,-0.014361,2021-05-20 06:09:00,2021-05-20 06:34:00,0 days 00:25:00
...,...,...,...,...,...,...,...,...,...,...
51,-493,17120,17127,2114.144004,2088.90,12445.293972,0.011941,2021-05-23 07:19:00,2021-05-23 07:26:00,0 days 00:07:00
60,409,19244,19252,2589.845524,2622.87,13507.010684,0.012752,2021-05-24 18:43:00,2021-05-24 18:51:00,0 days 00:08:00
34,342,13132,13138,2877.870688,2915.45,12852.124704,0.013058,2021-05-20 12:51:00,2021-05-20 12:57:00,0 days 00:06:00
21,-301,9246,9250,3268.002276,3224.09,13217.595076,0.013437,2021-05-17 20:05:00,2021-05-17 20:09:00,0 days 00:04:00


### Volume

In [222]:
%%time
stats, heatmap = bt.optimize(VOLUME_THRESHOLD = range(10,51, 5), VOLUME_LENGTH = range(5,30,5),maximize='Return [%]', return_heatmap=True)

CPU times: user 8.54 s, sys: 310 ms, total: 8.85 s
Wall time: 4min 16s


In [223]:
stats

Start                     2021-05-10 18:26:00
End                       2021-07-09 21:27:00
Duration                     60 days 03:01:00
Exposure Time [%]                    3.810261
Equity Final [$]               1330508.270492
Equity Peak [$]                1336385.616648
Return [%]                          33.050827
Buy & Hold Return [%]              -38.561976
Return (Ann.) [%]                  452.169477
Volatility (Ann.) [%]              149.350666
Sharpe Ratio                         3.027569
Sortino Ratio                       39.961602
Calmar Ratio                        66.872482
Max. Drawdown [%]                   -6.761667
Avg. Drawdown [%]                   -0.683897
Max. Drawdown Duration       11 days 22:57:00
Avg. Drawdown Duration        0 days 12:19:00
# Trades                                  293
Win Rate [%]                        69.624573
Best Trade [%]                       3.103065
Worst Trade [%]                     -3.894395
Avg. Trade [%]                    

In [224]:
stats._strategy

<Strategy AroonPullback(VOLUME_THRESHOLD=15,VOLUME_LENGTH=10)>

In [227]:
plot_heatmaps(heatmap, agg='mean')

Column(id='27283', ...)

In [240]:
%%time
stats, heatmap = bt.optimize(NATR_THRESHOLD_MR = range(25,50), maximize='Return [%]', return_heatmap=True)

CPU times: user 8.16 s, sys: 258 ms, total: 8.42 s
Wall time: 2min 14s


In [241]:
stats

Start                     2021-05-10 18:26:00
End                       2021-07-09 21:27:00
Duration                     60 days 03:01:00
Exposure Time [%]                     2.27761
Equity Final [$]               1385201.169704
Equity Peak [$]                1386351.279704
Return [%]                          38.520117
Buy & Hold Return [%]              -38.561976
Return (Ann.) [%]                  602.679389
Volatility (Ann.) [%]              189.422904
Sharpe Ratio                         3.181661
Sortino Ratio                       57.839099
Calmar Ratio                       112.615199
Max. Drawdown [%]                    -5.35167
Avg. Drawdown [%]                   -0.593624
Max. Drawdown Duration       11 days 12:51:00
Avg. Drawdown Duration        0 days 10:54:00
# Trades                                  184
Win Rate [%]                        72.826087
Best Trade [%]                       3.103065
Worst Trade [%]                     -3.894395
Avg. Trade [%]                    

In [242]:
stats._strategy

<Strategy AroonPullback(NATR_THRESHOLD_MR=37)>

In [244]:
heatmap.sort_values()

NATR_THRESHOLD_MR
49                   20.732755
48                   20.904645
47                   22.832319
44                   25.338384
46                   26.246553
45                   26.277803
42                   26.992108
43                   27.416506
26                   29.347494
25                   29.555945
31                   29.635665
40                   30.704656
41                   30.867042
34                   32.013344
32                   32.443833
28                   32.503853
33                   32.686558
30                   33.050827
39                   33.669472
29                   33.955246
27                   34.045613
38                   35.659607
35                   38.174935
36                   38.446360
37                   38.520117
Name: Return [%], dtype: float64

### SMA to regress

In [256]:
%%time
stats, heatmap = bt_2.optimize(SMA_LENGTH = range(7,20), return_heatmap=True)

CPU times: user 9.92 s, sys: 331 ms, total: 10.2 s
Wall time: 1min 45s


In [257]:
stats

Start                     2021-05-11 09:59:00
End                       2021-07-10 13:00:00
Duration                     60 days 03:01:00
Exposure Time [%]                    4.983715
Equity Final [$]               1345736.006364
Equity Peak [$]                 1366693.65576
Return [%]                          34.573601
Buy & Hold Return [%]              -46.615276
Return (Ann.) [%]                  478.305864
Volatility (Ann.) [%]              191.513219
Sharpe Ratio                         2.497508
Sortino Ratio                       29.096883
Calmar Ratio                        48.163927
Max. Drawdown [%]                    -9.93079
Avg. Drawdown [%]                   -0.704388
Max. Drawdown Duration       28 days 00:28:00
Avg. Drawdown Duration        0 days 11:21:00
# Trades                                  380
Win Rate [%]                        65.789474
Best Trade [%]                       1.904153
Worst Trade [%]                     -2.706334
Avg. Trade [%]                    

In [258]:
stats._strategy

<Strategy AroonPullback(SMA_LENGTH=15)>

In [259]:
heatmap.sort_values()

SMA_LENGTH
7             0.271209
8             1.056024
10            1.537594
12            1.627269
13            1.739705
11            1.872095
9             1.888154
14            2.092869
18            2.118915
16            2.149423
17            2.163639
19            2.272422
15            2.439231
Name: SQN, dtype: float64

## Do trades overlap between btc and eth?

In [281]:
trades_btc

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,type
0,-19,3069,3075,50433.588492,49944.47,9293.251348,0.009698,2021-05-12 21:35:00,2021-05-12 21:41:00,0 days 00:06:00,0
1,-19,3158,3175,50569.744008,50504.89,1232.226152,0.001282,2021-05-12 23:04:00,2021-05-12 23:21:00,0 days 00:17:00,0
2,-20,3274,3290,50441.805204,50800.00,-7163.895920,-0.007101,2021-05-13 01:00:00,2021-05-13 01:16:00,0 days 00:16:00,0
3,-19,3650,3655,50171.173500,49887.73,5385.426500,0.005650,2021-05-13 07:16:00,2021-05-13 07:21:00,0 days 00:05:00,0
4,-20,3725,3737,49611.937284,49584.19,554.945680,0.000559,2021-05-13 08:31:00,2021-05-13 08:43:00,0 days 00:12:00,0
...,...,...,...,...,...,...,...,...,...,...,...
179,38,71437,71442,35415.910700,35722.66,11656.473400,0.008661,2021-06-29 09:03:00,2021-06-29 09:08:00,0 days 00:05:00,0
180,-41,74497,74508,33409.880700,33439.00,-1193.891300,-0.000872,2021-07-01 12:03:00,2021-07-01 12:14:00,0 days 00:11:00,0
181,-40,81526,81533,34143.097296,33992.57,6021.091840,0.004409,2021-07-06 09:12:00,2021-07-06 09:19:00,0 days 00:07:00,0
182,39,82712,82717,34648.593896,34792.07,5595.568056,0.004141,2021-07-07 04:58:00,2021-07-07 05:03:00,0 days 00:05:00,0


In [282]:
trades_eth

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,type
0,-240,1682,1695,4160.115288,4170.82,-2569.130880,-0.002573,2021-05-12 14:01:00,2021-05-12 14:14:00,0 days 00:13:00,1
1,-242,1863,1872,4104.837408,4075.45,7111.752736,0.007159,2021-05-12 17:02:00,2021-05-12 17:11:00,0 days 00:09:00,1
2,-252,2189,2193,3978.727872,3942.26,9189.903744,0.009166,2021-05-12 22:28:00,2021-05-12 22:32:00,0 days 00:04:00,1
3,-257,2341,2352,3939.243672,3952.69,-3455.706296,-0.003413,2021-05-13 01:00:00,2021-05-13 01:11:00,0 days 00:11:00,1
4,-270,2792,2798,3733.945824,3713.50,5520.372480,0.005476,2021-05-13 08:31:00,2021-05-13 08:37:00,0 days 00:06:00,1
...,...,...,...,...,...,...,...,...,...,...,...
248,617,73801,73813,2113.875212,2109.22,-2872.265804,-0.002202,2021-07-01 16:00:00,2021-07-01 16:12:00,0 days 00:12:00,1
249,-573,78790,78808,2269.391880,2283.27,-7952.162760,-0.006115,2021-07-05 03:09:00,2021-07-05 03:27:00,0 days 00:18:00,1
250,-585,79499,79511,2211.125196,2218.65,-4402.010340,-0.003403,2021-07-05 14:58:00,2021-07-05 15:10:00,0 days 00:12:00,1
251,554,80403,80411,2324.929600,2335.00,5579.001600,0.004331,2021-07-06 06:02:00,2021-07-06 06:10:00,0 days 00:08:00,1


In [288]:
total_trades = pd.concat([trades_btc, trades_eth])
len(total_trades)

437

In [290]:
total_trades_sorted = total_trades.sort_values('EntryTime')

In [294]:
total_trades_sorted['next_trade_time'] = total_trades_sorted['EntryTime'].shift(-1)

In [301]:
total_trades_sorted

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,type,next_trade_time,OverlapsTime
0,-240,1682,1695,4160.115288,4170.82,-2569.130880,-0.002573,2021-05-12 14:01:00,2021-05-12 14:14:00,0 days 00:13:00,1,2021-05-12 17:02:00,0
1,-242,1863,1872,4104.837408,4075.45,7111.752736,0.007159,2021-05-12 17:02:00,2021-05-12 17:11:00,0 days 00:09:00,1,2021-05-12 21:35:00,0
0,-19,3069,3075,50433.588492,49944.47,9293.251348,0.009698,2021-05-12 21:35:00,2021-05-12 21:41:00,0 days 00:06:00,0,2021-05-12 22:28:00,0
2,-252,2189,2193,3978.727872,3942.26,9189.903744,0.009166,2021-05-12 22:28:00,2021-05-12 22:32:00,0 days 00:04:00,1,2021-05-12 23:04:00,0
1,-19,3158,3175,50569.744008,50504.89,1232.226152,0.001282,2021-05-12 23:04:00,2021-05-12 23:21:00,0 days 00:17:00,0,2021-05-13 01:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,554,80403,80411,2324.929600,2335.00,5579.001600,0.004331,2021-07-06 06:02:00,2021-07-06 06:10:00,0 days 00:08:00,1,2021-07-06 09:12:00,0
181,-40,81526,81533,34143.097296,33992.57,6021.091840,0.004409,2021-07-06 09:12:00,2021-07-06 09:19:00,0 days 00:07:00,0,2021-07-06 22:38:00,0
252,559,81399,81412,2316.436204,2315.18,-702.218036,-0.000542,2021-07-06 22:38:00,2021-07-06 22:51:00,0 days 00:13:00,1,2021-07-07 04:58:00,0
182,39,82712,82717,34648.593896,34792.07,5595.568056,0.004141,2021-07-07 04:58:00,2021-07-07 05:03:00,0 days 00:05:00,0,2021-07-07 09:43:00,0


In [304]:
sum(total_trades_sorted['OverlapsTime'])

95

In [296]:
total_trades_sorted['OverlapsTime'] = 0

In [297]:
total_trades_sorted.loc[total_trades_sorted['next_trade_time'] < total_trades_sorted['ExitTime'], 'OverlapsTime'] = 1

In [309]:
print('Total de trades overlapeados:', sum(total_trades_sorted['OverlapsTime'])/len(total_trades_sorted), '%')

Total de trades overlapeados: 0.21739130434782608 %


In [312]:
total_trades_sorted['OverlapsStart'] = 0

In [313]:
total_trades_sorted.loc[total_trades_sorted['next_trade_time'] == total_trades_sorted['EntryTime'], 'OverlapsStart'] = 1

In [314]:
print('Total de trades que comienzan al mismo momento:', sum(total_trades_sorted['OverlapsStart'])/len(total_trades_sorted), '%')

Total de trades que comienzan al mismo momento: 0.12356979405034325 %
